# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: ** Classificação. A resposta não é contínua, contendo apenas valores discretos 'yes' e 'no'.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [1]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [2]:
# TODO: Calcule o número de estudante
n_students = student_data.shape[0]

# TODO: Calcule o número de atributos
n_features = student_data.shape[1]

# TODO: Calcule o número de alunos aprovados
n_passed = student_data["passed"].value_counts().yes

# TODO: Calcule o número de alunos reprovados
n_failed = student_data["passed"].value_counts().no

# TODO: Calcule a taxa de graduação
grad_rate = 100*n_passed/(n_passed + n_failed)

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 31
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.00%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [12]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [13]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [14]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.model_selection import train_test_split
# TODO: Estabeleça o número de pontos de treinamento
num_train = 300.0

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=(num_test/(num_train+num_test)), random_state=0)


# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **
## Gaussian Naive Bayes:
- Aplicação:  
    Classificação de texto. Devido à sua ampla utilização em problemas multi-classe, é possível construir sistemas de recomendação, análise de sentimento e outras aplicações utilizando informações de texto.
- Vantagens:  
    Predições em tempo real, NB é realmente rápido para realizar predições.  
    Bom Algoritmo para predições de mais de uma classe.
    Capacidade de performar melhor que outros modelos como regressão logística, sem precisar de tantos dados
    
- Desvantagens:  
    Não é possível realizar a predição, quando surge uma nova categoria em uma variável categórica nos dados de teste, em relação aos de treinamento
    NB é considerado um mal estimador. A probabilidade estimada pelo classificador não deve ser levada em conta em alguns casos
    A  suposição de independência não é valida na maioria dos problemas reais.
- Por que escolher?  
    O NB é útil para ter uma base. Como o NB é rápido, é interessante utilizar ele. Se for válido para o problema, continue com ele. Caso contrário, é necessário buscar alguma alternativa que resulte nmuma performance melhor.

https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
## Gradient Boosting:
- Aplicação:  
    Prevenir fraudes em um sistema de compras online. o Alto poder do GB é capaz de generalizar bem os dados, uma vez que combina a informação de várias árvores de decisão. Algoritmos de Boosting são amplamente utilizados no mercado. 
- Vantagens:  
    Improvável overfit, combinação de modelos individiduais no treinamento tendem a não haver overfit. Acombinação destes modelos tendem a prevenir overfit.
    Redução de variância. O possível ruído no treinamento de um modelo só é atenuado pela combinação de resultados de um conjunto de modelos. 
    Controle do viés. Modelos enviesádos em média podem ter seus erros diminuídos.
- Desvantagens:
    Tempo alto de processamento para treinar.
    Modelos com alta variância podem não ser resolvidos com esse tipo de esemble, seria melhor utilizar bagging. 
- Por que escolher?
    GB pode ter uma performance boa, identificando padrões com menor problema com seu enviesamento.  
    
https://quantdare.com/what-is-the-difference-between-bagging-and-boosting/
## Máquinas de vetores de suporte:
- Aplicação:
    detecção de caracteres manuscritos em imagens. Devido ao alto poder de classificação do SVM, padrões não lineares como formas de imagens podem ser detectados.
- Vantagens:  
    Possibilidade de detectar padrões nos dados que não podem ser linearmente separados, utilizando um kernel não linear.
    Garantia de otimização, devido à natureza da otimização convexa.
    
- Desvantagens:
    Difícil interpretação dos resultados
    Treinamento pode ser muito demorado
    Não funciona bem com classes sobrepostas, ou com ruído.
- Por que escolher?  
    SVMs podem detectar poadrões não lineares. Como o Conjunto de dados não é grande suficiente para inviabilizar o modelo, devido à sua complexidade no tempo, é uma boa forma de ter um modelo diferente dos descritos acima.
    
https://pt.wikipedia.org/wiki/M%C3%A1quina_de_vetores_de_suporte
https://www.hackingnote.com/en/machine-learning/algorithms-pros-and-cons/
https://www.quora.com/What-are-some-pros-and-cons-of-Support-Vector-Machines

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [15]:
def grifar(text):
    return '\x1b[1;37;40m' + str(text) + '\x1b[0m'

In [16]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print  "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__,(len(X_train)) )  
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino:" +grifar(" {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print "Pontuação F1 para o conjunto de teste:"+grifar(" {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [22]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

# TODO: Inicialize os três modelos

clf_A = GaussianNB() # sem random state
clf_B = GradientBoostingClassifier(random_state=0)
clf_C = SVC(random_state=0)

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
classifiers = [clf_A,clf_B,clf_C]

for clf in classifiers:
    train_predict(clf, X_train_100, y_train_100, X_test, y_test)
    print '---//---'
    train_predict(clf, X_train_200, y_train_200, X_test, y_test)
    print '---//---'
    train_predict(clf, X_train_300, y_train_300, X_test, y_test)
    print '\n\n\n'

    

Treinando um GaussianNB com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0010 segundos
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.8550.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7481.
---//---
Treinando um GaussianNB com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0030 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8321.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7132.
---//---
Treinando um GaussianNB com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8088.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7500.




Treinando um GradientBoostingClassifier com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0490 segundos
A

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - GaussianNB**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |      0.0020           |        0.0010              |       0.8550               |  0.7481              |
| 200                                |       0.0030          |        0.0010              |         0.8321             |             0.7132         |
| 300                                |       0.0030          |       0.0030                |        0.8088              |        0.7500       |

** Classificador 2 - GradientBoostingClassifier **  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |     0.1120           |          0.0000             |     1.0000             |    0.7761           |
| 200                                |       0.1080        |         0.0020            |        0.9852                |   0.7820                   |
| 300                                |       0.1210          |          0.0010             |           0.9740           |        0.7727       |

** Classificador 3 - SVC **  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |    0.0100            |       0.0000                      |         0.8591          | 0.7838                    |
| 200                                |       0.0050         |        0.0020           |           0.8693        |       0.7820         |
| 300                                |    0.0100           |         0.0030              |          0.8692            |              0.7586        |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **
O modelo mais apropriado é o SVC. Ele apresentou um F1 score muito semelhante ao GradientBoosting, ambos superiores ao GaussianNB. Apresentou um tempo de treinamento inferior ao GB, tornando-o melhor escolha entre os três algoritmos selecionados. O tempo de estimativa dos três algoritmos foram próximos, na mesma ordem de grandeza.
A escolha do SVC pode ser explicada por sua métrica de avaliação F1 score, se tratando de uma base de dados de treinamento pequena o suficiente para o algoritmo não demorar tanto para treinar, uma vez que o SVC tem complexidade mais que quadratica, o tempo de treinamento poderia ser muito grande, com uma base de dados maior, inviabilizando sua aplicação [1].

[1]http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

Dúvida: Os valores de tempo de processamento não alteraram muito, qual o motivo? eu esperaria que o SVC fosse mais demorado, por exemplo. Como para o de 100 para 200 amostras, o tempo de treinamento pode diminuir?

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **
O modelo escolhido é o SVC. Modelo caracterizado por utilizar vetores de suporte, com objetivo de separar os dados com a margem maior possível. A estimativa do modelo é feita em cima da separação realizada no treino, onde cada amostra nova será classificada em uma das classes,aprovado ou reprovado, dependendo da distância entre as classes já separadas. Utilizando uma base de dados pequena como esta, o tempo utilizado pelo modelo não prejudica sua aplicação.

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [32]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3, 1e-4,1e-5],
                     'C': [ 0.1,1,10, 100,1000]}]

# TODO: Inicialize o classificador
clf = SVC(random_state=0)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(fbeta_score, beta=1,pos_label = 'yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, parameters, cv=5, scoring = f1_scorer )

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj = grid_obj.fit(X_train, y_train)


# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

As previsões foram feitas em 0.0120 segundos.
O modelo calibrado tem F1 de 0.8316 no conjunto de treinamento.
As previsões foram feitas em 0.0020 segundos.
O modelo calibrado tem F1 de 0.7862 no conjunto de teste.


## Dúvida
Por que é preciso utilizar  o parâmetro pos_label na função fbeta_score?

### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **
O modelo final calibrado obeteve, para treinamento, f1-score de 0.8316, e 0.7862 para teste.
Os resultado apresentou melhora de 3% em relação ao não calibrado para 300 amostras de treino no parâmetro F1. Esse aumento foi resultado da busca em grade, escolhendo os parâmetros que obtiveram melhor resultado F1, Esse método pode ser muito custoso computacionalmente, utilizando o modelo SVC, porém para um conjunto de dados não tão grande, esse problema é compensado pela melhora na performance com parâmetros calibrados.

Dúvida: Existe alguma forma de calibrar o modelo mais eficiente utilizando bibliotecas como sklearn mais utilizada?

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.